In [140]:
import pickle
import re
import pandas as pd
import string

#notes1 = pickle.load(open("pickle/notes1-100000.p", "rb" ))
notes2 = pickle.load(open("pickle/notes100000-200000.p", "rb" ))
notes3 = pickle.load(open("pickle/notes200000-.p", "rb" ))
notes = notes2 + notes3

links = pickle.load(open("pickle/links.p", "rb" ))

print(len(notes))
print(len(links))

129336
285557


In [141]:
def process_abc_note(note):
    lines = note.strip().splitlines()
    title = [x for x in lines if re.compile("T:").match(x)]
    title = title[0].split(":")[1].strip() if len(title)>0 else None

    time = [x for x in lines if re.compile("M:").match(x)]
    time = time[0].split(":")[1].strip() if len(time)>0 else None

    length = [x for x in lines if re.compile("L:").match(x)]
    length = length[0].split(":")[1].strip() if len(length)>0 else None

    tune_type = [x for x in lines if re.compile("R:").match(x)]
    tune_type = tune_type[0].split(":")[1].strip() if len(tune_type)>0 else None

    keyindex = [i for i, x in enumerate(lines) if re.compile("K:").match(x)]
    key = lines[keyindex[0]].split(":")[1].strip() if len(keyindex)>0 else None
    keyindex = 0 if len(keyindex)==0 else keyindex[0]

    tune = [x for x in lines[keyindex:] if re.compile(".*\\|").match(x)]
    tune = "".join(tune) if len(tune)>0 else None
    
    barsearch = re.compile(":?\|+:?\]?")
    bars = None
    if tune is not None:
        bars = [bar for bar in barsearch.split(tune) if bar!=""]
        barlines = barsearch.findall(tune)
        bars = [b+l for b, l in zip(bars,barlines)]

    features = {
        "title": title,
        "time": time,
        "length": length,
        "tune_type": tune_type,
        "key": key,
        "tune": tune,
        "bars": bars,
        "opening": None,
        "middle": None,
        "ending": None,
        "full_text": note
    }
    
    if bars is not None:
        features["opening"] = bars[:4]
        features["middle"] = bars[4:-4]
        features["ending"] = bars[-4:]
    
    features["complete"] = None not in features.values()
    return(features)

In [142]:
features = [process_abc_note(note) for note in notes]
# pickle.dump(features, open("pickle/features.p", "wb"))
len(features)

129336

In [143]:
complete_features = [x for x in features if x['complete']]

# pickle.dump(complete_features, open("pickle/complete_features.p", "wb"))
len(complete_features)

68630

In [144]:
df = pd.DataFrame(complete_features)

In [145]:
def parse_key(key):
    # if key is defined
    if len(key) == 0:
        return 'parsing failed'
    elif len(key) < 1:
        return(key)
    elif key[0] in string.ascii_uppercase[:7]:
        rootkey = re.compile("^[A-G][b#]?").search(key)
        rootkey = rootkey.group(0)
        keystem = key[len(rootkey):].lower().strip()
        major = bool(re.search("maj", keystem))
        minor = bool(re.search("min", keystem)) or bool(re.search("aeo", keystem))
        mix = bool(re.search("mix", keystem))
        dor = bool(re.search("dor", keystem))
        phr = bool(re.search("phr", keystem))
        lyd = bool(re.search("lyd", keystem))
        loc = bool(re.search("loc", keystem))
        if not major and not mix and len(keystem)>0 and keystem[0] == "m":
            minor = True
        key = rootkey
        if minor:
            key = key + "m"
        if mix:
            key = key + "Mix"
        if dor:
            key = key + "Dor"
        if phr:
            key = key + "Phr"
        if lyd and not mix:
            key = key + "Lyd"
        if loc:
            key = key + "Loc"
        return key
    else:
        if key.lower() == 'hp':
            return key
        elif key[0] == "^":
            return parse_key(key[1]+"#"+key[2:])
        elif key[0] == "_":
            return parse_key(key[1]+"b"+key[2:])
    return 'parsing failed'

In [146]:
df['key'] = df['key'].map(lambda k: parse_key(k))

In [139]:
df

,bars,complete,ending,full_text,key,length,middle,opening,time,title,tune,tune_type
0,"[F|, Eee^d e2 (BA)|, GBAF GEEF|, Ddd^c d2 (AG)...",True,"[G2 BG F2AF|, Eeef gefd|, BcdB AGFA|, BGAF GEE||]",\nX: 47\nT: Hatton Burn\nT: Tarbolton Lodge\nR...,Em,1/8,"[FdAG FDDF|, Eee^d efga|, fedf eBBA|, GABG FGA...","[F|, Eee^d e2 (BA)|, GBAF GEEF|, Ddd^c d2 (AG)|]",C,Hatton Burn,F|Eee^d e2 (BA)|GBAF GEEF|Ddd^c d2 (AG)|FdAG F...,Reel
1,"[F|, Eee^d e2 (BA)|, GBAF GEEF|, Ddd^c d2 (AG)...",True,"[G2 BG F2AF|, Eeef gefd|, BcdB AGFA|, BGAF GEE||]",\nX:1\n%\nT:Hatton Burn [1]\nT:Tarbolton Lodge...,Em,1/8,"[FdAG FDDF|, Eee^d efga|, fedf eBBA|, GABG FGA...","[F|, Eee^d e2 (BA)|, GBAF GEEF|, Ddd^c d2 (AG)|]",C,Hatton Burn [1],F|Eee^d e2 (BA)|GBAF GEEF|Ddd^c d2 (AG)|FdAG F...,Reel
2,"[D<D A2 ~B>A F<A|, D<D A2 ~B>A F<d|, D<D A2 ~B...",True,"[d<df>e d>A F<A|, D>AF>A .G/.F/.E/.D/ F<A|, d/...",\nX:1\n%\nT:Hattonburn [2]\nC:J. Scott Skinner...,D,1/8,"[2 A,<A,C>E A>EC>E||, d<d f>e d>A F<A|, .D.A.F...","[D<D A2 ~B>A F<A|, D<D A2 ~B>A F<d|, D<D A2 ~B...",C,Hattonburn [2],D<D A2 ~B>A F<A|D<D A2 ~B>A F<d|D<D A2 ~B>A F<...,"""Dancing Strathspey"""
3,"[=E |, \Ff-f.=d eEGB |, Ff-fe c=def |, gef=...",True,"[ ffee |, ffcf |, eee=e |, fcff/ |]]",\nI:abc-charset utf-8\n\nX:3003\nT:Hatton of B...,Fm,1/8,"[ cABG !trill!F2 {=E}F :|, g |, \afcf eEGg |,...","[=E |, \Ff-f.=d eEGB |, Ff-fe c=def |, gef=...",C|,Hatton of Buchromb's Reel,=E | \Ff-f.=d eEGB | Ff-fe c=def | gef=d eEGB ...,Reel
4,"[=E|:, Ff f=d eE GB|, Ff fe c=d ef|, ge f=d eE...",True,"[af cf eE Gg|, af cf =ef cf|, ga bg eE GB|, cA...",\nX:1\nT: Hatton of Buchromb's Reel\nC:William...,Fm,1/8,"[cA BG F2 F=E|, Ff f=d eE GB|, Ff fe c=d ef|, ...","[=E|:, Ff f=d eE GB|, Ff fe c=d ef|, ge f=d eE...",4/4,Hatton of Buchromb's Reel,|:=E|Ff f=d eE GB|Ff fe c=d ef|ge f=d eE GB|cA...,Reel
5,"[=E |, \Ff-f.=d eEGB |, Ff-fe c=def |, gef=...",True,"[ ffee |, ffcf |, eee=e |, fcff/ |]]",\nX:3549\nT:Hatton of Buchromb's Reel\nB:Wm Ma...,Fm,1/8,"[ cABG !trill!F2 {=E}F :|, g |, \afcf eEGg |,...","[=E |, \Ff-f.=d eEGB |, Ff-fe c=def |, gef=...",C|,Hatton of Buchromb's Reel,=E | \Ff-f.=d eEGB | Ff-fe c=def | gef=d eEGB ...,Reel
6,"[Bc|, d2B2G2E2|, c2A2F2D2|, G2D2A2D2|, GA B2A2...",True,"[d2d2d2d2|, d6c2|, B2G2 A2F2|, G6:||]",\nX:1\nT:Hatton Wall\nM:C|\nL:1/8\nR:Country D...,G,1/8,"[GA B2A2D2|, d2B2G2E2|, c2A2F2D2|, d2B2G2e2|, ...","[Bc|, d2B2G2E2|, c2A2F2D2|, G2D2A2D2|]",C|,Hatton Wall,Bc|d2B2G2E2|c2A2F2D2|G2D2A2D2|GA B2A2D2|d2B2G2...,Country Dance
7,"[ |, G3FG2 |, d3cd2 |, _e2c2d2 |, _B4A2 |,...",True,"[ z4d2 |, _e2d2c2 |, _B2A2G2 |, A3GA2 |]",\nX:296\nT: HAT UNS NICHT MAHOMET SCHAENDLICH ...,G,1/16,"[ _B4A2 |, B3AG2 |, _B2A2GA |, _B2B2c2 |, d...","[ |, G3FG2 |, d3cd2 |, _e2c2d2 |]",3/8,HAT UNS NICHT MAHOMET SCHAENDLICH BETROGEN,| G3FG2 | d3cd2 | _e2c2d2 | _B4A2 |B3AG2 | _B...,"Trink - Lied, national, chauvinistisch"
8,"[ Adfd e2ee |:, edfd e2e2 \|, 1,3 B{c}BBd cde...",True,"[1,3 B2Bd cde2 |, edcB A2A2 \|, 2,4 B2Bd cded...","\nX: 1\nT: Haudanmaa ja peli-Jussi\nO: trad, f...",D,1/8,"[2,4 B2Bd cded :|, [d4F4] [d4F4] |, fefa gfg...","[ Adfd e2ee |:, edfd e2e2 \|, 1,3 B{c}BBd cde...",4/4,Haudanmaa ja peli-Jussi,"|: Adfd e2ee | edfd e2e2 \|1,3 B{c}BBd cde2 | ...","march, reel"
9,"[ Adfd e2ee |:, edfd e2e2 |, 1,3 B{c}BBd cde2...",True,"[1,3 G2GB ABc2-|, cBAG E4 |, 2,4 G2GB A{B}AGF...","\nX: 1\nT: Haudanmaa ja peli-Jussi\nO: trad, f...",D,1/8,"[2,4 B2Bd cded :|, [d4F4] [d4F4] |, fefa gfg...","[ Adfd e2ee |:, edfd e2e2 |, 1,3 B{c}BBd cde2...",4/4,Haudanmaa ja peli-Jussi,"|: Adfd e2ee | edfd e2e2 |1,3 B{c}BBd cde2 | e...","march, reel"
